In [71]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
from pylab import rcParams
from pathlib import Path
from scipy import stats
import seaborn as sns
from sklearn import preprocessing
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [72]:
root = Path('/Users','alexandreberkovic','Desktop', 'Year_4','SIoT','IoT')

In [73]:
df = pd.read_csv(root / 'Output_data/CSV/sensor.csv')
humidity = pd.read_csv(root / 'Output_data/CSV/humidity.csv')
temperature = pd.read_csv(root / 'Output_data/CSV/temperature.csv')
api = pd.read_csv(root / 'Output_data/CSV/api.csv')

In [74]:
api.head()

,Duration,In,Out,Light Sleep,Deep Sleep,REM Sleep,Score,BPM,Awake,Interruptions,TTS,Date
0,436,2021-11-11 00:30:00,2021-11-11 08:21:00,258,134,44,85,57,28,5,11,2021-11-11
1,376,2021-11-12 00:54:00,2021-11-12 07:53:00,185,123,68,72,56,43,1,13,2021-11-12
2,431,2021-11-13 00:10:00,2021-11-13 08:25:00,211,155,65,86,59,57,4,41,2021-11-13
3,399,2021-11-14 00:14:00,2021-11-14 07:50:00,198,123,78,80,53,55,2,48,2021-11-14
4,412,2021-11-14 23:38:00,2021-11-15 08:04:00,224,120,68,77,58,94,3,52,2021-11-15


In [75]:
humidity.columns

Index(['Mean', 'Median', 'Std', 'Variance', 'Min', 'Max', 'Range', 'Date'], dtype='object')

In [76]:
humidity.corrwith(temperature, axis = 0)

Mean        0.995113
Median      0.995201
Std         0.919059
Variance    0.970156
Min         0.994614
Max         0.993632
Range       0.977940
dtype: float64

In [77]:
humidity.columns = ['h_avg', 'h_med', 'h_std', 'h_var', 'h_min', 'h_max', 'h_range', 'Date']
temperature.columns = ['t_avg', 't_med', 't_std', 't_var', 't_min', 't_max', 't_range', 'Date']

In [78]:
df = pd.concat([api, temperature], axis=1)

In [79]:
# df = pd.concat([df, temperature], axis=1)

In [80]:
df = df.dropna()

In [81]:
df = df.drop('Date',axis = 1)

In [82]:
df = df.drop(['In', 'Out'],axis = 1)

In [85]:
df = df.drop(['t_med', 't_var'],axis = 1)

In [86]:
df.head()

,Duration,Light Sleep,Deep Sleep,REM Sleep,Score,BPM,Awake,Interruptions,TTS,t_avg,t_std,t_min,t_max,t_range
0,436,258,134,44,85,57,28,5,11,19.454348,0.816417,18.9,21.3,2.4
1,376,185,123,68,72,56,43,1,13,19.541176,0.694601,19.0,21.1,2.1
2,431,211,155,65,86,59,57,4,41,19.292105,0.977139,18.6,21.2,2.6
3,399,198,123,78,80,53,55,2,48,18.236000,1.063064,16.8,21.6,4.8
4,412,224,120,68,77,58,94,3,52,18.580000,0.197122,18.3,19.0,0.7


In [87]:
corrM = df.corr()
corrM.style.background_gradient(cmap='coolwarm',axis=None).set_precision(2)
# plt.savefig('correlation.png')
# corrM

,Duration,Light Sleep,Deep Sleep,REM Sleep,Score,BPM,Awake,Interruptions,TTS,t_avg,t_std,t_min,t_max,t_range
Duration,1.00,0.79,0.78,0.52,0.96,-0.61,0.16,0.43,0.20,-0.13,-0.22,-0.09,-0.14,-0.12
Light Sleep,0.79,1.00,0.37,0.02,0.72,-0.40,0.20,0.65,0.09,0.20,-0.05,0.23,0.18,-0.05
Deep Sleep,0.78,0.37,1.00,0.41,0.75,-0.40,0.15,0.18,0.27,-0.27,-0.19,-0.30,-0.29,-0.03
REM Sleep,0.52,0.02,0.41,1.00,0.57,-0.56,-0.09,-0.15,0.09,-0.42,-0.32,-0.34,-0.41,-0.23
Score,0.96,0.72,0.75,0.57,1.00,-0.66,-0.03,0.34,0.08,-0.18,-0.26,-0.12,-0.19,-0.18
BPM,-0.61,-0.40,-0.40,-0.56,-0.66,1.00,0.04,0.06,-0.15,0.35,0.26,0.35,0.34,0.05
Awake,0.16,0.20,0.15,-0.09,-0.03,0.04,1.00,0.24,0.83,0.31,0.21,0.18,0.29,0.30
Interruptions,0.43,0.65,0.18,-0.15,0.34,0.06,0.24,1.00,-0.10,0.50,0.03,0.61,0.47,-0.16
TTS,0.20,0.09,0.27,0.09,0.08,-0.15,0.83,-0.10,1.00,0.13,0.32,-0.07,0.15,0.48
t_avg,-0.13,0.20,-0.27,-0.42,-0.18,0.35,0.31,0.50,0.13,1.00,0.64,0.91,1.00,0.39


In [70]:
# Fill diagonal and upper half with NaNs
mask = np.zeros_like(corrM, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corrM[mask] = np.nan
(corrM
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

,Duration,Light Sleep,Deep Sleep,REM Sleep,Score,BPM,Awake,Interruptions,TTS,h_avg,h_std,h_min,h_max,h_range
Duration,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Light Sleep,0.79,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Deep Sleep,0.78,0.37,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
REM Sleep,0.52,0.02,0.41,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Score,0.96,0.72,0.75,0.57,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
BPM,-0.61,-0.40,-0.40,-0.56,-0.66,nan,nan,nan,nan,nan,nan,nan,nan,nan
Awake,0.16,0.20,0.15,-0.09,-0.03,0.04,nan,nan,nan,nan,nan,nan,nan,nan
Interruptions,0.43,0.65,0.18,-0.15,0.34,0.06,0.24,nan,nan,nan,nan,nan,nan,nan
TTS,0.20,0.09,0.27,0.09,0.08,-0.15,0.83,-0.10,nan,nan,nan,nan,nan,nan
h_avg,-0.08,0.23,-0.22,-0.40,-0.14,0.31,0.35,0.49,0.19,nan,nan,nan,nan,nan


In [38]:
api.corrwith(humidity)

Awake           NaN
BPM             NaN
Deep Sleep      NaN
Duration        NaN
Interruptions   NaN
Light Sleep     NaN
REM Sleep       NaN
Score           NaN
TTS             NaN
h_avg           NaN
h_max           NaN
h_med           NaN
h_min           NaN
h_range         NaN
h_std           NaN
h_var           NaN
dtype: float64

In [531]:
# time stamp column to ts instead of string
df['Time stamp'] = pd.to_datetime(df['Time stamp'], infer_datetime_format=True)

In [532]:
# # remove humidity levels below 30 (as those should not occur)
df = df.loc[df['Humidity'] >= 30]
# df = df.reset_index()

In [533]:
df = df.loc[df['Light'] >= 0]

In [534]:
df = df.set_index('Time stamp')

In [535]:
humidity = df['Humidity']
temperature = df['Temperature']
light = df['Light']

In [536]:
# resampling at every 10min 
# h = humidity.resample('10min').mean()

In [537]:
# y = y[3058:]

In [538]:
h = humidity.resample('10min').mean()
t = temperature.resample('10min').mean()
l = light.resample('10min').mean()

In [539]:
# yNew = y.fillna(method="ffill")
# yNew = y.interpolate('akima').dropna()
h = h.interpolate('akima').dropna()
t = t.interpolate('akima').dropna()
l = l.interpolate('akima').dropna()

In [540]:
# h = h[3500:3700]
# t = t[3500:3700]
# l = l[3500:3700]

In [541]:
h.sort_index(inplace=True)
t.sort_index(inplace=True)
l.sort_index(inplace=True)
yNew.sort_index(inplace=True)

In [502]:
h_val = h.values.reshape(-1, 1) #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
h_scaled = min_max_scaler.fit_transform(h_val)
h_norm = pd.DataFrame(h_scaled).set_index(h.index)
h_norm.columns = ['Humidity']